In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import yfinance as yf
import datetime

# Download stock data
def get_stock_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    return data

# Set parameters
ticker = 'AAPL'  # Apple stock
start_date = '2015-01-01'
end_date = datetime.datetime.now().strftime('%Y-%m-%d')
time_steps = 60  # Number of previous days to use for prediction

# Download the data
df = get_stock_data(ticker, start_date, end_date)
print(f"Downloaded {len(df)} days of {ticker} data")
df.head()

# Plot the closing price history
plt.figure(figsize=(16, 8))
plt.title(f'{ticker} Closing Price History')
plt.plot(df['Close'])
plt.xlabel('Date', fontsize=14)
plt.ylabel('Close Price USD ($)', fontsize=14)
plt.show()

# Prepare the data for training
data = df['Close'].values.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Split into train and test sets (80% train, 20% test)
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size - time_steps:]

# Create dataset with sequences for LSTM
def create_dataset(data, time_steps):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:(i + time_steps), 0])
        y.append(data[i + time_steps, 0])
    return np.array(X), np.array(y)

X_train, y_train = create_dataset(train_data, time_steps)
X_test, y_test = create_dataset(test_data, time_steps)

# Reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)

# Plot training history
plt.figure(figsize=(16, 8))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Make predictions
train_predictions = model.predict(X_train)
test_predictions = model.predict(X_test)

# Inverse transform the predictions to get actual prices
train_predictions = scaler.inverse_transform(train_predictions)
test_predictions = scaler.inverse_transform(test_predictions)
y_train_actual = scaler.inverse_transform(y_train.reshape(-1, 1))
y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))

# Calculate RMSE
train_rmse = np.sqrt(mean_squared_error(y_train_actual, train_predictions))
test_rmse = np.sqrt(mean_squared_error(y_test_actual, test_predictions))
print(f'Train RMSE: {train_rmse:.2f}')
print(f'Test RMSE: {test_rmse:.2f}')

# Visualize the predictions
# Create a dataframe for actual vs predicted values
train_data_len = len(train_data)
test_dates = df.index[train_size:train_size+len(test_predictions)]

# Plot for test predictions
plt.figure(figsize=(16, 8))
plt.title(f'{ticker} Actual vs Predicted Prices')
plt.plot(df.index[train_size:train_size+len(y_test_actual)], y_test_actual, label='Actual Prices')
plt.plot(df.index[train_size:train_size+len(test_predictions)], test_predictions, label='Predicted Prices')
plt.xlabel('Date')
plt.ylabel('Price USD ($)')
plt.legend()
plt.show()

# Predict future values (next 30 days)
last_60_days = scaled_data[-time_steps:]
X_future = []
X_future.append(last_60_days)
X_future = np.array(X_future)
X_future = np.reshape(X_future, (X_future.shape[0], X_future.shape[1], 1))

# Predict the next day
future_price = model.predict(X_future)
future_price = scaler.inverse_transform(future_price)
print(f"Predicted price for next trading day: ${future_price[0][0]:.2f}")

# Make predictions for the next 30 days
future_predictions = []
current_batch = last_60_days

for i in range(30):
    # Get prediction for next day
    current_pred = model.predict(np.reshape(current_batch, (1, time_steps, 1)))
    
    # Add the prediction to our list
    future_predictions.append(current_pred[0][0])
    
    # Update the batch to include the new prediction
    current_batch = np.append(current_batch[1:], current_pred)
    current_batch = current_batch.reshape(time_steps, 1)

# Inverse transform to get actual prices
future_predictions = np.array(future_predictions).reshape(-1, 1)
future_predictions = scaler.inverse_transform(future_predictions)

# Create future dates
last_date = df.index[-1]
future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=30, freq='B')

# Plot future predictions
plt.figure(figsize=(16, 8))
plt.title(f'{ticker} Future Price Predictions')
plt.plot(df.index[-100:], df['Close'].values[-100:], label='Historical Prices')
plt.plot(future_dates, future_predictions, label='Predicted Prices')
plt.xlabel('Date')
plt.ylabel('Price USD ($)')
plt.legend()
plt.show()

print("Disclaimer: This model is for educational purposes only. Stock market predictions are inherently uncertain.")